README: https://docs.google.com/document/d/1cxqU0d6jLgmjK3pTIK_nI7M9fo7NDUt3qGktD8knNZ8/edit  

API Details:  
https://min-api.cryptocompare.com/  
https://www.cryptocompare.com/api  

Code Ref:  
https://github.com/MaverickLLC/cryptocompareapi_python_MavericLLC/blob/master/cryptocompare_maverick.py  
https://github.com/lagerfeuer/cryptocompare/blob/master/cryptocompare/cryptocompare.py  

TO DO:  
- Make generic functions for tasks for modularity
- Make a class for individual coin if required
- Incorporate https://github.com/mrjbq7/ta-lib

In [1]:
# Imports
import time
import logging
import requests
import datetime
import talib
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict

In [2]:
# API
API_ENDPOINT = "https://min-api.cryptocompare.com/data"
OLD_API_ENDPOINT = "https://www.cryptocompare.com/api/data"

URL_COIN_LIST = OLD_API_ENDPOINT + "/coinlist/"
URL_COIN_SNAPSHOT = OLD_API_ENDPOINT + "/coinsnapshot/?fsym={}&tsym={}&e={}"

URL_PRICE = API_ENDPOINT + "/pricemulti?fsyms={}&tsyms={}&e={}"
URL_PRICE_MULTI = API_ENDPOINT + "/pricemulti?fsyms={}&tsyms={}&e={}"
URL_PRICE_MULTI_FULL = API_ENDPOINT + "/pricemultifull?fsyms={}&tsyms={}&e={}"

URL_HIST_PRICE = API_ENDPOINT + "/pricehistorical?fsym={}&tsyms={}&ts={}"
URL_HIST_PRICE_DAY = API_ENDPOINT + "/histoday?fsym={}&tsym={}&e={}&limit={}&allData={}"
URL_HIST_PRICE_HOUR = API_ENDPOINT + "/histohour?fsym={}&tsym={}&e={}&limit={}"
URL_HIST_PRICE_MINUTE = API_ENDPOINT + "/histominute?fsym={}&tsym={}&e={}&toTs={}"

URL_AVG = API_ENDPOINT + "/generateAvg?fsym={}&tsym={}&e={}"
URL_DAY_AVG = API_ENDPOINT + "/dayAvg?fsym={}&tsym={}&e={}"

In [3]:
# FIELDS
PRICE = 'PRICE'
HIGH = 'HIGH24HOUR'
LOW = 'LOW24HOUR'
VOLUME = 'VOLUME24HOUR'
CHANGE = 'CHANGE24HOUR'
CHANGE_PERCENT = 'CHANGEPCT24HOUR'
MARKETCAP = 'MKTCAP'
NPERIODS = 100
TIMEFRAME = 'Day'

In [4]:
# Defaults
CURR = "USD"
EXCHANGE = "CCCAGG"
COIN = 'BTC'
COIN_LIST = ['BTC', 'ETH']
EXCHANGES = ['Bittrex','Binance']

In [5]:
def query_cryptocompare(url):
    """ Query CryptoCompare API """
    try:
        response = requests.get(url).json()
    except Exception as e:
        logging.error("Failure while querying {query}. \n{err}".format(query=url, err=e))
        return None
    return response

In [6]:
# Helper functions
def format_parameter(parameter):
    """ Format parameters for the query """
    if isinstance(parameter, list):
        return ','.join(parameter)
    else:
        return parameter
    
def convert_timestamp(timestamp):
    return datetime.datetime.fromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')

def get_data(response):
    """ Separate query data from response """
    header = {key: (value if key != 'Data' else len(value)) for key, value in response.items()}
    data = response['Data']
    return header, data

def get_readable_df(response):
    """ Convert timestamp into readable datetime """
    header, data = get_data(response)
    df_data = pd.DataFrame(data)
    df_data = df_data.rename(columns={'time': 'timestamp'})
    df_data['time'] = df_data.timestamp.apply(convert_timestamp)
    df_data = df_data.set_index('time')
    del df_data['timestamp']
    return df_data

In [7]:
def get_coin_list():
    """ Get coin list """
    return query_cryptocompare(URL_COIN_LIST)['Data']

In [8]:
coins = get_coin_list()
COIN_DB = pd.DataFrame.from_dict(coins, orient='index')

In [9]:
def get_price(coin, to_curr=CURR, exchange=EXCHANGE):
    """ Get real time price of the coin """
    if isinstance(coin, list):
        return query_cryptocompare(URL_PRICE_MULTI.format(
                format_parameter(coin), format_parameter(to_curr), exchange
            )
        )
    else:
        return query_cryptocompare(URL_PRICE.format(coin, format_parameter(to_curr), exchange))

In [10]:
get_price(COIN_LIST)

{'BTC': {'USD': 10850.64}, 'ETH': {'USD': 853.59}}

In [11]:
def get_historical_price_timestamp(coin, to_curr=CURR, timestamp=time.time(), exchange=EXCHANGE):
    """ Get value of coin in currency at a particular timestamp """
    if isinstance(timestamp, datetime.datetime):
        timestamp = time.mktime(timestamp.timetuple())
    
    return query_cryptocompare(URL_HIST_PRICE.format(
            coin, 
            format_parameter(to_curr),
            int(timestamp),
            exchange
        )
    )

In [12]:
get_historical_price_timestamp(COIN)

{'BTC': {'USD': 10813.06}}

In [13]:
def get_historical_price_day(coin, to_curr=CURR, timestamp=time.time(), exchange=EXCHANGE, limit=30, allData='false'):
    """ Get price per day for the past month """
    return query_cryptocompare(URL_HIST_PRICE_DAY.format(
            coin,
            format_parameter(to_curr),
            exchange,
            limit,
            allData
        )
    )

def get_historical_price_last_day(*args):
    """ Get price for last day """
    return get_historical_price_day(*args, limit=1)

In [ ]:
# coin_day = get_historical_price_day(COIN)
# df_coin_day = get_readable_df(coin_day)
coin_day = get_historical_price_day('ZRX','BTC',exchange='Binance',limit=1)
if coin_day['Response'] != 'Error':
    df_coin_day = get_readable_df(coin_day)

In [14]:
def get_historical_price_day_full(*args):
    """ Get price per day for all time  """
    return get_historical_price_day(*args, allData='true')

In [ ]:
coin_day_full = get_historical_price_day_full(COIN)
df_coin_day_full = get_readable_df(coin_day_full)

In [15]:
def get_historical_price_hour(coin, to_curr=CURR, exchange=EXCHANGE, limit=168):
    """ Get price per hour for past 7 days """
    return query_cryptocompare(URL_HIST_PRICE_HOUR.format(
            coin,
            format_parameter(to_curr),
            exchange,
            limit
        )
    )

def get_historical_price_last_hour(*args, **kwargs):
    """ Get price for the last hour """
    return get_historical_price_hour(*args, **kwargs, limit=1)

In [ ]:
coin_hour = get_historical_price_hour(COIN)
df_coin_hour = get_readable_df(coin_hour)

In [16]:
def get_historical_price_minute(coin, to_curr=CURR, exchange=EXCHANGE, toTs=time.time()):
    """ Get price per min for past 24 hours """
    return query_cryptocompare(URL_HIST_PRICE_MINUTE.format(
            coin,
            format_parameter(to_curr),
            exchange,
            int(toTs)
        )
    )

In [ ]:
coin_min = get_historical_price_minute(COIN)
df_coin_min = get_readable_df(coin_min)

In [17]:
def get_historical_price_minute_by_day(*args, days_ago=0):
    """ Get price per min for 24 hours till days_ago """
    if days_ago > 7:
        logging.error("Can not get information by minute for more than 7 days. Getting information for last possible day.")
        days_ago = 7
    days_ago -= 1  # Subtracting one day as toTs considers ending timestamp
    ts = datetime.datetime.today() - datetime.timedelta(days_ago)
    ts = time.mktime(ts.timetuple())
    return get_historical_price_minute(*args, toTs=int(ts))

In [ ]:
coin_minute_week = get_historical_price_minute_by_day(COIN, days_ago=7)
df_coin_minute_week = get_readable_df(coin_minute_week)

-----------------------

In [18]:
# Data
csv_day_full = 'coin_day_full.csv'

In [19]:
df_coin_day_full['coin'] = 'BTC'
df_coin_day_full.to_csv(csv_day_full)

NameError: name 'df_coin_day_full' is not defined

In [53]:
price_data={}
# message, data = Mapi.getHistoDay('ZRX', to_symbol = 'BTC')
df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
# print(data[0]['volumefrom'])
i=0
j=0
coins_set_day = set()
coins_set_min = set()
coins_set_hour = set()
got_all = False
for exchange in EXCHANGES :    # For all exchanges
    if got_all == True:
        break
    for symbol in COIN_DB.Symbol:         #For every coin we get in linr number 16 in this code
       #if len(coins_set) == 2:
           #break
       if symbol not in coins_set_day :
           data = get_historical_price_day(symbol, 'BTC', time.time(),exchange,20)  
            # Get it's minute/hour/daily/weekly data
           if data['Response'] != 'Error' : 
               df_coin_day_all = get_readable_df(data)
               df_coin_day_all['coin']=symbol
               df_coin_day_all['exchange']=exchange 
               df = df.append(df_coin_day_all) 
               #print(df_coin_day_all)
               coins_set_day.add(symbol)
       if len(coins_set_day) == 5:
            got_all = True
            break
                       
# print(df)
csv_all_coins_day_full = 'all_coins_day_full.csv'
df.to_csv(csv_all_coins_day_full)

In [20]:
def fetch_last_day(coin):
    """ Get data for last day """
    coin_last_day = get_historical_price_last_day(COIN)
    df_coin_last_day = get_readable_df(coin_last_day)
    df_coin_last_day['coin'] = coin
    last_entry = df_coin_last_day.iloc[-1:]
    return last_entry

In [21]:
def update_last_day(coin):
    """ Update csv for last day """
    record_last_day = fetch_last_day(COIN)
    csv_day = 'coin_day_full.csv'
    df_day = pd.read_csv(csv_day)
    record = df_day[df_day['timestamp'] == record_last_day.iloc[0]['timestamp']]
    if record.empty:
        record_last_day.to_csv(csv_day, mode='a', header=False)

In [ ]:
from functools import partial
update_last_day_partial = partial(update_last_day, COIN)

In [ ]:
from apscheduler.schedulers.background import BackgroundScheduler
scheduler = BackgroundScheduler({'apscheduler.timezone': 'UTC'})
# Update data every day at 5:30pm
scheduler.add_job(update_last_day_partial, 'cron', day_of_week='*', hour=17, minute=30)
scheduler.start()

In [22]:
# Maps csv (future data objects) to period granularity
# If we store all data together in a single data source, we'll change this to a function which returns corresponding rows
data_csv_period_mapping = {
    "day": 'all_coins_day_full.csv',
    "hour": 'coin_hour.csv',
    "min": 'coin_min.csv'
}

In [23]:
def fetch_data(coin_symbol=['BTC'], nrows=1, period='day'):
    """ Fetch data from csv for mentioned coin 
        This function fetches last nrows from csv corr to given period
    """
    period = period.lower()
    csv_filename = data_csv_period_mapping[period]
    df_csv = pd.read_csv(csv_filename, index_col=None)
    
    return_df = []
    for coin in coin_symbol:
        # Get rows where 'coin' is one of the coin_symbols; then extract last nrows
        req_data = df_csv[df_csv['coin'] == coin].iloc[-nrows:]
        req_columns = ['coin', 'time', 'open', 'high', 'low', 'close', 'volumeto']
        req_data = req_data[req_columns]
        return_df.append(req_data)
    return return_df
    # return pd.concat(return_df)

In [25]:
NPERIODS = 20
data = fetch_data(['1ST', 'ABY'], NPERIODS, TIMEFRAME)
# print(data)
btc_data, eth_data = data

In [26]:
print(type(btc_data))
#print(eth_data)
btc_data.shape

<class 'pandas.core.frame.DataFrame'>


(20, 7)

In [28]:
from talib import abstract
from talib.abstract import *
inp_data = btc_data
inp_data_two = eth_data
inp_sma = SMA(inp_data, timeperiod=3)
inp_sma_two = SMA(inp_data_two,timeperiod=5)
inp_rsi = talib.RSI(inp_data.close.values, timeperiod=14)
inp_mfi = talib.MFI(inp_data.high.values, inp_data.low.values, inp_data.close.values, inp_data.volumeto.values, timeperiod=14)
inp_bbands = talib.BBANDS(inp_data.close.values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

In [54]:
inp_data['SMA'] = inp_sma
inp_data_two['SMA'] = inp_sma_two

In [55]:
def update_indicator(csv_filename, indicator_data, indicator):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None)
    df_csv = pd.merge(df_csv,
                      indicator_data[['coin', 'time', indicator]],
                      on=['coin', 'time'],
                      how='left')
#     df_csv = df_csv.set_index('time')
    # return df_csv
    df_csv.to_csv(csv_filename, index=False)

In [56]:
update_indicator('all_coins_day_full.csv', inp_data, 'SMA')
update_indicator('all_coins_day_full.csv', inp_data_two,'SMA')